In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import psycopg2

In [4]:
db_connect = psycopg2.connect(
  database="reccomendation_system", 
  user="Aleksandr", 
  password="qweasd", 
  host="127.0.0.1", 
  port="5432"
)

OperationalError: could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "127.0.0.1" and accepting
	TCP/IP connections on port 5432?
